In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from bayes_opt import BayesianOptimization


# Load and prepare data
file_path = '/Users/schoudhry/Desktop/shashy/traderAuto/wmntJan9.csv'
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)


In [2]:

# Constants
initial_capital = 1000
initial_price = 84
shares_bought = initial_capital / initial_price

# Define trading simulation function for optimizer
def trading_profit(buy_threshold, sell_threshold):
    cash, equity = 0, shares_bought
    for _, row in df.iterrows():
        price = row['Open']
        if price <= buy_threshold and cash > price:
            qty = cash // price
            cash -= qty * price
            equity += qty
        elif price >= sell_threshold and equity >= 1:
            cash += equity * price
            equity = 0
    final_value = cash + equity * df.iloc[-1]['Close']
    return final_value - initial_capital

# Parameter bounds for optimizer
pbounds = {
    'buy_threshold': (488.0, 495.0),
    'sell_threshold': (500.5, 514.0)
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(
    f=trading_profit,
    pbounds=pbounds,
    random_state=42,
    verbose=2  # Display iteration details
)

optimizer.maximize(init_points=5, n_iter=25)

# Best threshold values
best_params = optimizer.max['params']
best_buy = best_params['buy_threshold']
best_sell = best_params['sell_threshold']

# Simulate with best thresholds
cash, equity = 0, shares_bought
buys = sells = holds = 0
trade_log = []

for _, row in df.iterrows():
    price = row['Open']
    if price <= best_buy and cash > price:
        qty = cash // price
        cash -= qty * price
        equity += qty
        action = f"BUY {qty} @ {price:.2f}"
        buys += 1
    elif price >= best_sell and equity >= 1:
        cash += equity * price
        action = f"SELL {equity} @ {price:.2f}"
        equity = 0
        sells += 1
    else:
        action = "HOLD"
        holds += 1
    trade_log.append({
        'Date': row['Date'],
        'Action': action,
        'Cash': round(cash, 2),
        'Equity': round(equity, 4),
        'Price': price
    })

# Final value and summary
final_value = cash + equity * df.iloc[-1]['Close']
profit = final_value - initial_capital

# Summary Table
bayes_summary = pd.DataFrame([{
    'Strategy': 'Bayesian Optimization',
    'Buy Threshold': round(best_buy, 2),
    'Sell Threshold': round(best_sell, 2),
    'Initial Capital': initial_capital,
    'Buys': buys,
    'Sells': sells,
    'Holds': holds,
    'Final Value': round(final_value, 2),
    'Profit': round(profit, 2)
}])

# Trade Log Table
trade_log_df = pd.DataFrame(trade_log)



# Save or print
print(bayes_summary)
print("\nTop of Trade Log:")
print(trade_log_df.head())
bayes_summary.to_csv("bayes_summary.csv", index=False)
trade_log_df.to_csv("bayes_trade_log.csv", index=False)



|   iter    |  target   | buy_th... | sell_t... |
-------------------------------------------------
| 1         | 187.7     | 490.6     | 513.3     |
| 2         | 187.7     | 493.1     | 508.6     |
| 3         | 187.7     | 489.1     | 502.6     |
| 4         | 187.7     | 488.4     | 512.2     |
| 5         | 187.7     | 492.2     | 510.1     |
| 6         | 187.7     | 494.9     | 500.5     |
| 7         | 187.7     | 495.0     | 500.5     |
| 8         | 187.7     | 488.3     | 514.0     |
| 9         | 187.7     | 488.0     | 500.5     |
| 10        | 187.7     | 495.0     | 514.0     |
| 11        | 187.7     | 495.0     | 500.5     |
| 12        | 187.7     | 491.1     | 511.6     |
| 13        | 187.7     | 488.0     | 500.6     |
| 14        | 187.7     | 490.8     | 507.1     |
| 15        | 187.7     | 492.8     | 502.2     |
| 16        | 187.7     | 492.2     | 502.5     |
| 17        | 187.7     | 495.0     | 514.0     |
| 18        | 187.7     | 488.1     | 513.9     |
